# Comparison with Trucost Data

In [6]:
#!pip install pycryptodome

import pandas as pd
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup
import json
import io
from PyPDF2 import PdfReader
import random

ImportError: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with LibreSSL 2.8.3. See: https://github.com/urllib3/urllib3/issues/2168

In [1]:
#import data from private computer beacuse we cannot store trucost data on github
path_data = r'/Users/computer/Documents/2023/EPFL/ml/assignments/assignment3/data/'
df_performance = pd.read_csv(path_data + r'companies_environmental-performance.csv')
df_sustainability_report = pd.read_excel(path_data + 'companies_sustainability_report.xlsx')

NameError: name 'pd' is not defined

In [3]:
df_sustainability_report.rename(columns={'LINK TO DOWNLOAD THE SELECTED REPORT': 'REPORT_LINK'}, inplace=True)
df_sustainability_report.rename(columns={'ISIN_CODE': 'ISIN'}, inplace=True)
df_sustainability_report.rename(columns={'COMPANY_NAME': 'COMPANY'}, inplace=True)
df_performance.rename(columns={'GICS Sector Name': 'sector_name'}, inplace=True)


In [4]:
df_performance

,TCUID,Company,ISIN,Financial Year,GICS Sector Code,sector_name,GICS Industry Group Code,GICS Industry Group Name,GICS Industry Code,GICS Industry Name,...,Natural Resource Use-Indirect Impact Ratio (%),Natural Resource Use-Total Impact Ratio (%),Carbon Disclosure,Carbon-Weighted Disclosure (%),Water-Weighted Disclosure (%),Waste-Weighted Disclosure (%),Revenue (USD mn),Accounting Year End,FigureRestate,EffectiveDate
0,53174,SHUAA Capital PSC,AE0005802527,2018,40,Financials,4020,Diversified Financials,402030,Capital Markets,...,0.013698,0.013698,Estimated data,0.0,NaN,0.0,37.046632,00:00.0,0,35:38.1
1,53174,SHUAA Capital PSC,AE0005802527,2017,40,Financials,4020,Diversified Financials,402010,Diversified Financial Services,...,0.013259,0.013259,Estimated data,0.0,NaN,0.0,40.261436,00:00.0,0,24:18.2
2,54147,Abu Dhabi Commercial Bank PJSC,AEA000201011,2020,40,Financials,4010,Banks,401010,Banks,...,0.010000,0.010000,Estimated data,0.0,NaN,0.0,2308.254756,00:00.0,0,51:49.0
3,54147,Abu Dhabi Commercial Bank PJSC,AEA000201011,2019,40,Financials,4010,Banks,401010,Banks,...,0.009980,0.009980,Derived from previous year,100.0,NaN,100.0,3668.168600,00:00.0,0,47:22.0
4,54147,Abu Dhabi Commercial Bank PJSC,AEA000201011,2018,40,Financials,4010,Banks,401010,Banks,...,0.010878,0.010878,Derived from previous year,100.0,NaN,100.0,2756.346473,00:00.0,0,46:45.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65282,227217,Simbisa Brands Limited,ZW0009012262,2021,25,Consumer Discretionary,2530,Consumer Services,253010,"Hotels, Restaurants & Leisure",...,0.072000,0.072000,Estimated data,0.0,NaN,100.0,58.291592,00:00.0,0,09:56.9
65283,227217,Simbisa Brands Limited,ZW0009012262,2020,25,Consumer Discretionary,2530,Consumer Services,253010,"Hotels, Restaurants & Leisure",...,0.069000,0.069000,Estimated data,0.0,NaN,0.0,29.081596,00:00.0,0,32:01.5
65284,227217,Simbisa Brands Limited,ZW0009012262,2019,25,Consumer Discretionary,2530,Consumer Services,253010,"Hotels, Restaurants & Leisure",...,0.069560,0.069560,Estimated data,0.0,NaN,0.0,1.211763,00:00.0,0,16:13.3
65285,227217,Simbisa Brands Limited,ZW0009012262,2018,25,Consumer Discretionary,2530,Consumer Services,253010,"Hotels, Restaurants & Leisure",...,0.075253,0.075253,Estimate used instead of disclosure - data is ...,0.0,NaN,0.0,204.728000,00:00.0,0,38:13.8


In [5]:
#chose firms which have available link to download selected report
df_sust_report_nonan = df_sustainability_report.dropna(subset=['REPORT_LINK'])
df_sust_report_nonan

,UNIQUE_FIRM_ID,COMPANY,ISIN,YEAR,UNIQUE_FIRM_YEAR_ID,TYPE_OF_REPORT,UNIQUE_FIRM_YEAR_REPORT_ ID,REPORT_LINK
0,241,GERRESHEIMER (XET),DE000A0LD6E6,2020.0,241_2020,AR,241_2020_AR,https://www.gerresheimer.com/fileadmin/user_up...
1,242,GETINGE B,SE0000202624,2020.0,242_2020,AR,242_2020_AR,https://www.getinge.com/dam/corporate/document...
2,242,GETINGE B,SE0000202624,2020.0,242_2020,SR,242_2020_SR,https://www.getinge.com/dam/corporate/document...
3,243,GETLINK,FR0010533075,2002.0,243_2002,AR,243_2002_AR,https://www.getlinkgroup.com/content/uploads/2...
4,244,GIVAUDAN 'N',CH0010645932,2020.0,244_2020,SR,244_2020_SR,https://integratedreport.givaudan.com/sites/gi...
...,...,...,...,...,...,...,...,...
446,581,VOLVO B,SE0000115446,2019.0,581_2019,AR,581_2019_AR,https://www.volvogroup.com/content/dam/volvo/v...
447,582,VONOVIA (XET),DE000A1ML7J1,2020.0,582_2020,AR,582_2020_AR,https://reports.vonovia.de/2020/annual-report/...
448,582,VONOVIA (XET),DE000A1ML7J1,2020.0,582_2020,SR,582_2020_SR,https://reports.vonovia.de/2020/sustainability...
449,582,VONOVIA (XET),DE000A1ML7J1,2019.0,582_2019,AR,582_2019_AR,https://reports.vonovia.de/2019/annual-report/...


In [6]:
#merge on ISIN using inner to only get the isins that are available in both
df_merged = df_sust_report_nonan.merge(df_performance, on='ISIN', how='inner')
df_merged

,UNIQUE_FIRM_ID,COMPANY,ISIN,YEAR,UNIQUE_FIRM_YEAR_ID,TYPE_OF_REPORT,UNIQUE_FIRM_YEAR_REPORT_ ID,REPORT_LINK,TCUID,Company,...,Natural Resource Use-Indirect Impact Ratio (%),Natural Resource Use-Total Impact Ratio (%),Carbon Disclosure,Carbon-Weighted Disclosure (%),Water-Weighted Disclosure (%),Waste-Weighted Disclosure (%),Revenue (USD mn),Accounting Year End,FigureRestate,EffectiveDate
0,241,GERRESHEIMER (XET),DE000A0LD6E6,2020.0,241_2020,AR,241_2020_AR,https://www.gerresheimer.com/fileadmin/user_up...,56571,Gerresheimer AG,...,0.209000,0.209000,Exact Value from CDP,99.942000,100.0,0.0,1604.807900,00:00.0,0,23:42.3
1,241,GERRESHEIMER (XET),DE000A0LD6E6,2020.0,241_2020,AR,241_2020_AR,https://www.gerresheimer.com/fileadmin/user_up...,56571,Gerresheimer AG,...,0.212269,0.212269,Exact Value from CDP,100.000000,100.0,0.0,1561.939100,00:00.0,0,05:23.7
2,241,GERRESHEIMER (XET),DE000A0LD6E6,2020.0,241_2020,AR,241_2020_AR,https://www.gerresheimer.com/fileadmin/user_up...,56571,Gerresheimer AG,...,0.227726,0.227726,Exact Value from CDP,99.814197,0.0,0.0,1619.590163,00:00.0,0,10:09.3
3,241,GERRESHEIMER (XET),DE000A0LD6E6,2020.0,241_2020,AR,241_2020_AR,https://www.gerresheimer.com/fileadmin/user_up...,56571,Gerresheimer AG,...,0.217561,0.217561,Exact Value from CDP,99.810550,0.0,0.0,1605.546400,00:00.0,0,29:58.8
4,242,GETINGE B,SE0000202624,2020.0,242_2020,AR,242_2020_AR,https://www.getinge.com/dam/corporate/document...,47867,Getinge AB (publ),...,0.040000,0.040000,Exact Value from Annual Report/10K/Financial A...,46.707000,100.0,100.0,3238.391009,00:00.0,0,51:07.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
646,582,VONOVIA (XET),DE000A1ML7J1,2019.0,582_2019,AR,582_2019_AR,https://reports.vonovia.de/2019/annual-report/...,189501,Vonovia SE,...,0.041631,0.041631,Exact Value from Environmental/CSR,100.000000,NaN,100.0,4320.363000,00:00.0,0,49:40.1
647,582,VONOVIA (XET),DE000A1ML7J1,2019.0,582_2019,SR,582_2019_SR,https://reports.vonovia.de/2019/sustainability...,189501,Vonovia SE,...,0.039000,0.039000,Value summed up from data provided in Environm...,100.000000,100.0,100.0,4594.373847,00:00.0,0,16:52.0
648,582,VONOVIA (XET),DE000A1ML7J1,2019.0,582_2019,SR,582_2019_SR,https://reports.vonovia.de/2019/sustainability...,189501,Vonovia SE,...,0.040000,0.040000,Value summed up from data provided in Environm...,100.000000,100.0,100.0,3258.476400,00:00.0,0,34:02.5
649,582,VONOVIA (XET),DE000A1ML7J1,2019.0,582_2019,SR,582_2019_SR,https://reports.vonovia.de/2019/sustainability...,189501,Vonovia SE,...,0.043000,0.043000,Value summed up from data provided in Environm...,100.000000,100.0,100.0,4757.110800,00:00.0,0,18:52.6


In [7]:
df_merged.keys()

Index(['UNIQUE_FIRM_ID', 'COMPANY', 'ISIN', 'YEAR', 'UNIQUE_FIRM_YEAR_ID',
       'TYPE_OF_REPORT', 'UNIQUE_FIRM_YEAR_REPORT_ ID', 'REPORT_LINK', 'TCUID',
       'Company', 'Financial Year', 'GICS Sector Code', 'sector_name',
       'GICS Industry Group Code', 'GICS Industry Group Name',
       'GICS Industry Code', 'GICS Industry Name', 'GICS Sub Industry Code',
       'GICS Sub Industry Name', 'GICS Description', 'Country',
       'Carbon-Scope 1  (tonnes CO2e)', 'Carbon-Scope 2  (tonnes CO2e)',
       'Carbon-Scope 3 (tonnes CO2e)', 'Carbon-Direct (tonnes CO2e)',
       'Carbon-First Tier Indirect (tonnes CO2e)',
       'Carbon-Direct+First Tier Indirect (tonnes CO2e)',
       'Water-Direct Cooling (cubic meter)',
       'Water-Direct Process (cubic meter)', 'Water-Purchased (cubic meter)',
       'Water-Direct+Purchased (cubic meter)', 'Waste-Landfill (tonnes)',
       'Waste-Incineration (tonnes)', 'Waste-Nuclear (tonnes)',
       'Waste-Recycled (tonnes)',
       'Carbon Intensit

In [24]:
df_merged.sector_name.unique()

array(['Health Care', 'Industrials', 'Materials', 'Financials',
       'Consumer Staples', 'Real Estate', 'Consumer Discretionary',
       'Information Technology', 'Utilities', 'Energy',
       'Communication Services'], dtype=object)

In [9]:
#choose financial sector
df_financials = df_merged.loc[df_merged['sector_name']=='Financials']

In [21]:
# function to check if link contains a pdf 
def check_pdf_link(link):
    try:
        response = requests.head(link)
        content_type = response.headers.get('content-type')
        if content_type == 'application/pdf':
            return True
    except requests.exceptions.RequestException:
        pass
    return False


In [ ]:
"""select 10 firms with random ISIN for which the link contain pdf files"""
# Set the random seed for reproducibility
random_seed = 3
random.seed(random_seed)

# Function to check if a given link is a PDF
def is_pdf_link(link):
    try:
        response = requests.head(link)
        content_type = response.headers.get('content-type')
        if content_type == 'application/pdf':
            return True
    except requests.exceptions.RequestException:
        pass
    return False

# Get unique ISIN codes from the DataFrame
financials_unique_isin = df_financials['ISIN'].unique()

# Remove any ISIN codes that are known to not have a PDF (e.g., 'GB00BD6GN030')
financials_unique_isin = financials_unique_isin[financials_unique_isin != 'GB00BD6GN030']

# Shuffle the unique ISIN codes
random.shuffle(financials_unique_isin)

# Select 10 firms with valid PDF links
selected_firms = []
for isin in financials_unique_isin:
    link = df_financials[df_financials['ISIN'] == isin]['REPORT_LINK'].iloc[0]
    if is_pdf_link(link):
        selected_firms.append(isin)
    if len(selected_firms) == 10:
        break

# Filter the DataFrame to include only the selected ISIN codes
financials_10_all_reports = df_financials[df_financials['ISIN'].isin(selected_firms)].copy()

# Access the PDF links or other desired information
fin_10_pdf_links = financials_10_all_reports['REPORT_LINK']


In [23]:
df_financials_pdf = df_financials[df_financials['REPORT_LINK'].apply(check_pdf_link)]

KeyboardInterrupt: 

In [10]:
#make reprodicible random selection of ISIN codes
random_seed = 3  # Set the random seed for reproducibility
random.seed(random_seed)

# Get unique ISIN codes from the DataFrame
financials_unique_isin = df_financials['ISIN'].unique()

#remove Virigin money cause their pdf is broken
financials_unique_isin2 = financials_unique_isin[financials_unique_isin != 'GB00BD6GN030']
# Shuffle the unique ISIN codes
random.shuffle(financials_unique_isin2)

# Select the first 10 unique ISIN codes
financials_selected_isin = financials_unique_isin2[:10]

# Filter the DataFrame to include only the selected ISIN codes
financials_10_all_reports = df_financials[df_financials['ISIN'].isin(financials_selected_isin)].copy()
fin_10_pdf = financials_10_all_reports[]

In [11]:
# Sort the selected entries by date in descending order
financials_10_all_reports.sort_values('YEAR', ascending=False, inplace=True)

# Keep only the first entry for each unique ISIN code
financials_10_newest_reports = financials_10_all_reports.groupby('ISIN').first().reset_index()


In [20]:
for index in range(0,10):
    print(financials_10_newest_reports.loc[index,'REPORT_LINK'])

https://www.gbl.be/en/media/3593/annual_report_2020.pdf
https://www.juliusbaer.com/index.php?eID=dumpFile&t=f&f=75197&token=26bcf52ad4576fb076e8ad0e0dcc85d0cb773599
https://reports.swissre.com/2020/
https://www.ubs.com/global/en/ubs-society/our-stories/2021/banking-on-sustainability.html
https://www.helvetia.com/content/dam/os/corporate/web/documents/corporate-responsibility/cr-report-helvetia-group-2020.pdf
https://www.munichre.com/content/dam/munichre/contentlounge/website-pieces/documents/CR-Report-2020.pdf/_jcr_content/renditions/original./CR-Report-2020.pdf
https://www.societegenerale.com/sites/default/files/documents/2021-07/Integrated-Report-2020-2021.pdf
https://www.fundslibrary.co.uk/FundsLibrary.DataRetrieval/Documents.aspx/?type=sl.ra.full&id=73305e94-1c5c-4b5f-b2ed-2785f8249bac&user=DbbcbsREkvWSk2DH3dLatpHiYDzVqxVQJZbueAKxizc9M5HhBM8a4ljN%2fcztqMqm&r=1
https://www.unicreditgroup.eu/en/a-sustainable-bank/sustainability-reporting.html
https://mb.cision.com/Public/14481/328561

In [16]:
df_financials.reset_index().loc[9,'REPORT_LINK',]

'https://www.gbl.be/system/files/public-files/RA2020%20(EN)%20-%20ESG%20section.pdf'

In [13]:
def read_pdf(url):
    """reads pdf from given url and returns the text as a string"""
    # download the PDF file from the URL
    response = requests.get(url)
    #print(response)
    
    # check if the request was successful (status code 200)
    if response.status_code == 200:
        print(f"Success: Request to {url} returned status code {response.status_code}")

        # read the PDF file into memory
        pdf_file = io.BytesIO(response.content)

        # create a PdfReader object and read the contents of the PDF file
        pdf_reader = PdfReader(pdf_file)

        # extract the text from each page of the PDF file
        text = ""
        for page_num in range(len(pdf_reader.pages)):
            text += pdf_reader.pages[page_num].extract_text()
        print('text import successfull')
        return text
    
    else:
        return ""

In [51]:
def read_pdf2(url):
    """reads pdf from given url and returns the text as a string"""
    try:
        # download the PDF file from the URL
        response = requests.get(url)
        
        # check if the request was successful (status code 200)
        if response.status_code == 200:
            print(f"Success: Request to {url} returned status code {response.status_code}")

            # read the PDF file into memory
            pdf_file = io.BytesIO(response.content)

            # create a PdfReader object and read the contents of the PDF file
            pdf_reader = PdfReader(pdf_file)

            # extract the text from each page of the PDF file
            text = ""
            for page_num in range(len(pdf_reader.pages)):
                text += pdf_reader.pages[page_num].extract_text()
            print('text import successfull')
            return text
    
        
    except requests.exceptions.RequestException as e:
        print(f"Warning: Error occurred while fetching {url}: {str(e)}")
        return ""


In [13]:
read_pdf(df_financials.reset_index().loc[0,'REPORT_LINK',])

' \n Gjensidige  Pensjonsforsikring  AS \n \n1  \n \n Gjensidige  Pensjonsforsikring  AS \n \n2 Annual report 2020 \n \nGjensidige Pensjonsforsikring AS (GPF) is a wholly owned \nsubsidiary of Gjensidige Forsikring ASA and is headquartered in \nOslo. Its activities are aimed at the life-  and pension business \nand the company offer products both to private and commercial \ncustomer s. The company was established in 2005 and focuses \non sales of defined contribution plans and risk coverage for both customer groups.  \nThe company delivers products in the following main categories:  \n• Occupational pension -  Defined contribution plans with \nassociated risk coverage  \n• Management of paid- up certificates and paid- up policies  \n• Individual pension savings (unit linked)  \n• Individual disability pension \n \nThe company offers only externally managed funds  for unit \nlinked products.  \nDistribution of the company\'s occupational products takes place \nprimarily through the pare

In [54]:
df_financials_reports = pd.DataFrame(columns=['ISIN', 'url', 'text'])

for index in range(6, 10):
    isin = financials_10_newest_reports.reset_index().loc[index, 'ISIN']
    url = financials_10_newest_reports.reset_index().loc[index, 'REPORT_LINK']
    text = read_pdf(url)
    df_financials_reports.loc[index] = [isin, url, text]

Success: Request to https://www.societegenerale.com/sites/default/files/documents/2021-07/Integrated-Report-2020-2021.pdf returned status code 200
text import successfull
Success: Request to https://www.virginmoneyukplc.com/resources/a19e641b-e11c-4af8-82b5-a0c6a9c80891/VMUKARA2021FINAL.pdf returned status code 200


PdfReadError: EOF marker not found

0     \n Gjensidige  Pensjonsforsikring  AS \n \n1 ...
Name: text, dtype: object